In [8]:
from pysmt.shortcuts import *
import pysmt.typing as types
import random as rn
from pysmt.typing import BOOL, REAL, INT, BVType, STRING
from IPython.display import Latex
import itertools

# Seção: Definição de Constantes
# Constantes que representam características físicas e limites do sistema.
atrito_restante_modos = 0.01  # Atrito nos modos FREE e STOPPING
atrito_modo_bloqueado = 0.009 # Atrito no modo BLOCKED
proporcionalidade_travagem_stopping = 10.0  # Proporcionalidade da força de travagem no modo STOPPING
proporcionalidade_travagem = 0.5  # Proporcionalidade da força de travagem nos modos FREE e BLOCKED
atrito_corpo_ar = 0.01 # Atrito devido ao contacto com o ar
peso_veiculo = 1000.0  # Peso do veículo
limite_timer = 0.1  # Limite máximo do contador nos modos FREE e BLOCKED
delta_tempo = 0.1  # Diferença de tempo nas transições temporizadas
velocidade_inicial = 20.0  # Velocidade inicial do veículo
margem_erro = 0.2  # Margem de erro para considerar velocidades como zero

# Seção: Definição de Modos
# Representação dos diferentes estados discretos do veículo.
MODO_INICIAL = Int(0)
MODO_LIVRE = Int(1)
MODO_TRAVAGEM = Int(2)
MODO_BLOQUEADO = Int(3)
MODO_PARADO = Int(4)

# Seção: Definição de Funções Principais

def declarar_variaveis(indice):
    """
    Declara as variáveis de estado para um determinado instante.

    Args:
        indice (int): O índice do instante de tempo.

    Returns:
        dict: Um dicionário contendo as variáveis de estado.
    """
    estado = {}
    estado['tempo'] = Symbol(f'tempo{indice}', REAL)  # Tempo decorrido
    estado['velocidade'] = Symbol(f'velocidade{indice}', REAL)  # Velocidade do veículo
    estado['velocidade_roda'] = Symbol(f'velocidade_roda{indice}', REAL)  # Velocidade da roda
    estado['modo'] = Symbol(f'modo{indice}', INT)  # Modo atual do veículo
    estado['temporizador'] = Symbol(f'temporizador{indice}', REAL)  # Contador de tempo no modo atual
    return estado

def inicializar_estado(estado, velocidade_inicial):
    """
    Define o estado inicial do sistema.

    Args:
        estado (dict): O dicionário de variáveis de estado inicial.
        velocidade_inicial (float): A velocidade inicial do veículo.

    Returns:
        FNode: Uma fórmula SMT que representa o estado inicial.
    """
    condicao_tempo = Equals(estado['tempo'], Real(0))
    condicao_velocidade = Equals(estado['velocidade'], Real(velocidade_inicial))
    condicao_velocidade_roda = Equals(estado['velocidade_roda'], Real(velocidade_inicial))
    condicao_modo = Equals(estado['modo'], MODO_INICIAL)
    condicao_temporizador = Equals(estado['temporizador'], Real(0))
    return And(condicao_tempo, condicao_velocidade, condicao_velocidade_roda, condicao_modo, condicao_temporizador)

def definir_transicao(estado_atual, proximo_estado):
    """
    Define as possíveis transições entre os estados do sistema.

    Args:
        estado_atual (dict): O dicionário de variáveis de estado atual.
        proximo_estado (dict): O dicionário de variáveis de estado no próximo instante.

    Returns:
        FNode: Uma fórmula SMT que representa as condições de transição.
    """
    # Transições não temporizadas
    transicao_inicial_livre = And(
        Equals(estado_atual['modo'], MODO_INICIAL),
        Equals(proximo_estado['modo'], MODO_LIVRE),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade']),
        Equals(proximo_estado['velocidade_roda'], estado_atual['velocidade_roda']),
        Equals(proximo_estado['tempo'], estado_atual['tempo']),
        Equals(proximo_estado['temporizador'], Real(0))
    )

    transicao_livre_travagem = And(
        Equals(estado_atual['modo'], MODO_LIVRE),
        Equals(proximo_estado['modo'], MODO_TRAVAGEM),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade']),
        Equals(proximo_estado['velocidade_roda'], estado_atual['velocidade_roda']),
        Equals(proximo_estado['tempo'], estado_atual['tempo']),
        Equals(proximo_estado['temporizador'], Real(0)),
        Or(estado_atual['temporizador'] >= limite_timer, And(estado_atual['velocidade'] <= margem_erro, estado_atual['velocidade_roda'] <= margem_erro))
    )

    transicao_travagem_parado = And(
        Equals(estado_atual['modo'], MODO_TRAVAGEM),
        Equals(proximo_estado['modo'], MODO_PARADO),
        Equals(proximo_estado['tempo'], estado_atual['tempo']),
        Equals(proximo_estado['temporizador'], Real(0)),
        And(estado_atual['velocidade_roda'] <= margem_erro, estado_atual['velocidade'] <= margem_erro, 
            estado_atual['velocidade_roda'] >= -margem_erro, estado_atual['velocidade'] >= -margem_erro),
        Equals(proximo_estado['velocidade'], Real(0)),
        Equals(proximo_estado['velocidade_roda'], Real(0))
    )

    transicao_travagem_bloqueado = And(
        Equals(estado_atual['modo'], MODO_TRAVAGEM),
        Equals(proximo_estado['modo'], MODO_BLOQUEADO),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade']),
        Equals(proximo_estado['tempo'], estado_atual['tempo']),
        Equals(proximo_estado['temporizador'], Real(0)),
        And(estado_atual['velocidade'] - estado_atual['velocidade_roda'] <= margem_erro, estado_atual['velocidade_roda'] - estado_atual['velocidade'] <= margem_erro),
        Equals(proximo_estado['velocidade_roda'], proximo_estado['velocidade']),
        Or(proximo_estado['velocidade'] > margem_erro, proximo_estado['velocidade_roda'] > margem_erro)
    )

    transicao_bloqueado_livre = And(
        Equals(estado_atual['modo'], MODO_BLOQUEADO),
        Equals(proximo_estado['modo'], MODO_LIVRE),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade']),
        Equals(proximo_estado['velocidade_roda'], estado_atual['velocidade_roda']),
        Equals(proximo_estado['tempo'], estado_atual['tempo']),
        Equals(proximo_estado['temporizador'], Real(0)),
        Or(estado_atual['temporizador'] >= limite_timer, estado_atual['velocidade'] <= margem_erro, estado_atual['velocidade_roda'] <= margem_erro)
    )

    # Transições temporizadas
    transicao_livre_livre = And(
        Equals(estado_atual['modo'], MODO_LIVRE),
        Equals(proximo_estado['modo'], MODO_LIVRE),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade'] + (-proporcionalidade_travagem * (estado_atual['velocidade'] - estado_atual['velocidade_roda']) - atrito_corpo_ar) * delta_tempo),
        Equals(proximo_estado['velocidade_roda'], estado_atual['velocidade_roda'] + (-atrito_restante_modos * peso_veiculo + proporcionalidade_travagem * (estado_atual['velocidade'] - estado_atual['velocidade_roda'])) * delta_tempo),
        Equals(proximo_estado['temporizador'], estado_atual['temporizador'] + delta_tempo),
        Equals(proximo_estado['tempo'], estado_atual['tempo'] + delta_tempo),
        proximo_estado['temporizador'] <= limite_timer,
        Or(proximo_estado['velocidade'] > margem_erro, proximo_estado['velocidade_roda'] > margem_erro)
    )

    transicao_travagem_travagem = And(
        Equals(estado_atual['modo'], MODO_TRAVAGEM),
        Equals(proximo_estado['modo'], MODO_TRAVAGEM),
        Equals(proximo_estado['temporizador'], Real(0)),
        Equals(proximo_estado['tempo'], estado_atual['tempo'] + delta_tempo),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade'] + (-proporcionalidade_travagem_stopping * (estado_atual['velocidade'] - estado_atual['velocidade_roda']) - atrito_corpo_ar) * delta_tempo),
        Equals(proximo_estado['velocidade_roda'], estado_atual['velocidade_roda'] + (-atrito_restante_modos * peso_veiculo + proporcionalidade_travagem_stopping * (estado_atual['velocidade'] - estado_atual['velocidade_roda'])) * delta_tempo),
        estado_atual['velocidade'] - estado_atual['velocidade_roda'] > margem_erro,
        proximo_estado['velocidade'] <= estado_atual['velocidade'],
        Or(estado_atual['velocidade'] > margem_erro, estado_atual['velocidade_roda'] > margem_erro)
    )

    transicao_parado_parado = And(
        Equals(estado_atual['modo'], MODO_PARADO),
        Equals(proximo_estado['modo'], MODO_PARADO),
        Equals(proximo_estado['temporizador'], Real(0)),
        Equals(proximo_estado['tempo'], estado_atual['tempo'] + delta_tempo),
        Equals(proximo_estado['velocidade'], Real(0)),
        Equals(proximo_estado['velocidade_roda'], Real(0))
    )

    transicao_bloqueado_bloqueado = And(
        Equals(estado_atual['modo'], MODO_BLOQUEADO),
        Equals(proximo_estado['modo'], MODO_BLOQUEADO),
        Equals(proximo_estado['tempo'], estado_atual['tempo'] + delta_tempo),
        Equals(proximo_estado['temporizador'], estado_atual['temporizador'] + delta_tempo),
        And(estado_atual['velocidade'] - estado_atual['velocidade_roda'] <= margem_erro, estado_atual['velocidade_roda'] - estado_atual['velocidade'] <= margem_erro),
        Equals(proximo_estado['velocidade'], estado_atual['velocidade'] + (-atrito_modo_bloqueado * peso_veiculo - atrito_corpo_ar) * delta_tempo),
        Equals(proximo_estado['velocidade_roda'], proximo_estado['velocidade']),
        proximo_estado['temporizador'] <= limite_timer,
        proximo_estado['velocidade'] > 0
    )

    return Or(transicao_inicial_livre, transicao_livre_livre, transicao_livre_travagem, transicao_travagem_travagem,
              transicao_travagem_parado, transicao_travagem_bloqueado, transicao_bloqueado_bloqueado,
              transicao_bloqueado_livre, transicao_parado_parado)

# Seção: Funções Auxiliares

def imprimir_variaveis(estado, solvedor):
    """
    Imprime os valores das variáveis de estado para um dado estado.

    Args:
        estado (dict): O dicionário de variáveis de estado.
        solvedor (Solver): O solvedor SMT utilizado.
    """
    print("MODO: ", end="")
    modo_valor = solvedor.get_py_value(estado['modo'])
    if modo_valor == 0:
        print("INICIAL")
    elif modo_valor == 1:
        print("LIVRE")
    elif modo_valor == 2:
        print("TRAVAGEM")
    elif modo_valor == 3:
        print("BLOQUEADO")
    elif modo_valor == 4:
        print("PARADO")
    print("Velocidade: ", end="")
    print(float(solvedor.get_py_value(estado['velocidade'])))
    print("Velocidade da Roda: ", end="")
    print(float(solvedor.get_py_value(estado['velocidade_roda'])))
    print("Tempo: ", end="")
    print(float(solvedor.get_py_value(estado['tempo'])))
    print("Temporizador: ", end="")
    print(float(solvedor.get_py_value(estado['temporizador'])))
    print("")
    print("")

def gerar_traco(declarar_variaveis, inicializar_estado, definir_transicao, k, velocidade_inicial):
    """
    Gera e imprime um traço de execução do sistema.

    Args:
        declarar_variaveis (function): Função para declarar variáveis de estado.
        inicializar_estado (function): Função para inicializar o estado.
        definir_transicao (function): Função para definir as transições.
        k (int): O número de estados no traço.
        velocidade_inicial (float): A velocidade inicial do veículo.
    """
    with Solver(name="z3") as solvedor:
        traco = [declarar_variaveis(i) for i in range(k)]
        solvedor.add_assertion(inicializar_estado(traco[0], velocidade_inicial))

        for i in range(k - 1):
            solvedor.add_assertion(definir_transicao(traco[i], traco[i + 1]))

        if solvedor.solve():
            i = 0
            print("is sat")
            for estado in traco:
                print("ESTADO: ", end="")
                print(i)
                imprimir_variaveis(estado, solvedor)
                i = i + 1
        else:
            print("unsat")

# Seção: Definição de Propriedades

def propriedade_limite_temporizador(estado):
    """
    Define a propriedade de que o temporizador não excede o limite nos modos FREE e BLOCKED.

    Args:
        estado (dict): O dicionário de variáveis de estado.

    Returns:
        FNode: Uma fórmula SMT que representa a propriedade.
    """
    return Implies(Or(Equals(estado['modo'], MODO_LIVRE), Equals(estado['modo'], MODO_BLOQUEADO)), estado['temporizador'] <= limite_timer)

def propriedade_paragem(estado):
    """
    Define a propriedade de que o veículo está parado.

    Args:
        estado (dict): O dicionário de variáveis de estado.

    Returns:
        FNode: Uma fórmula SMT que representa a propriedade.
    """
    return Equals(estado['modo'], MODO_PARADO)

def propriedade_velocidade_sempre_a_diminuir(estado_atual, proximo_estado):
    """
    Define a propriedade de que a velocidade do veículo diminui sempre com o tempo.

    Args:
        estado_atual (dict): O dicionário de variáveis de estado atual.
        proximo_estado (dict): O dicionário de variáveis de estado no próximo instante.

    Returns:
        FNode: Uma fórmula SMT que representa a propriedade.
    """
    return Implies(estado_atual['tempo'] <= proximo_estado['tempo'], proximo_estado['velocidade'] <= estado_atual['velocidade'])

# Seção: Funções de Verificação de Propriedades

def verificacao_kinducao_sempre(declarar_variaveis, inicializar_estado, definir_transicao, invariante, k, velocidade_inicial):
    """
    Verifica uma propriedade invariante usando k-indução.

    Args:
        declarar_variaveis (function): Função para declarar variáveis de estado.
        inicializar_estado (function): Função para inicializar o estado.
        definir_transicao (function): Função para definir as transições.
        invariante (function): Função que define o invariante a ser verificado.
        k (int): O valor de k para a k-indução.
        velocidade_inicial (float): A velocidade inicial do veículo.
    """
    with Solver(name="z3") as solvedor:
        estados = [declarar_variaveis(i) for i in range(k)]
        solvedor.add_assertion(inicializar_estado(estados[0], velocidade_inicial))
        for i in range(k - 1):
            solvedor.add_assertion(definir_transicao(estados[i], estados[i + 1]))

        for i in range(k):
            solvedor.push()
            solvedor.add_assertion(Not(invariante(estados[i])))
            if solvedor.solve():
                print(f"> Contradição! O invariante não se verifica nos {k} estados iniciais.")
                i = 0
                for estado in estados:
                    print("ESTADO: ", end="")
                    imprimir_variaveis(estado, solvedor)
                    i = i + 1
                return
            solvedor.pop()

        estados_seguintes = [declarar_variaveis(i + k) for i in range(k + 1)]

        for i in range(k):
            solvedor.add_assertion(invariante(estados_seguintes[i]))
            solvedor.add_assertion(definir_transicao(estados_seguintes[i], estados_seguintes[i + 1]))

        solvedor.add_assertion(inicializar_estado(estados_seguintes[0], velocidade_inicial))

        solvedor.add_assertion(Not(invariante(estados_seguintes[-1])))

        if solvedor.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            return

        print(f"> A propriedade verifica-se por k-indução (k={k}).")

def verificacao_bmc_sempre(declarar_variaveis, inicializar_estado, definir_transicao, invariante, K, velocidade_inicial):
    """
    Verifica uma propriedade invariante usando Bounded Model Checking (BMC).

    Args:
        declarar_variaveis (function): Função para declarar variáveis de estado.
        inicializar_estado (function): Função para inicializar o estado.
        definir_transicao (function): Função para definir as transições.
        invariante (function): Função que define o invariante a ser verificado.
        K (int): O limite para o BMC.
        velocidade_inicial (float): A velocidade inicial do veículo.
    """
    with Solver(name="z3") as solvedor:
        traco = [declarar_variaveis(i) for i in range(K)]
        solvedor.add_assertion(inicializar_estado(traco[0], velocidade_inicial))

        for k in range(K):
            if k > 0:
                solvedor.add_assertion(definir_transicao(traco[k - 1], traco[k]))

            solvedor.push()
            solvedor.add_assertion(Not(invariante(traco[k])))

            if solvedor.solve():
                print(f"> Invariante não se verifica nos primeiros {k + 1} estados")
            else:
                if k == K - 1:
                    print(f"> Invariante verifica-se nos {K} estados")
                else:
                    solvedor.pop()

def verificacao_bmc_eventualmente(declarar_variaveis, inicializar_estado, definir_transicao, propriedade, limite, velocidade_inicial):
    """
    Verifica se uma propriedade eventualmente ocorre dentro de um limite usando BMC.

    Args:
        declarar_variaveis (function): Função para declarar variáveis de estado.
        inicializar_estado (function): Função para inicializar o estado.
        definir_transicao (function): Função para definir as transições.
        propriedade (function): Função que define a propriedade a ser verificada.
        limite (int): O limite para o BMC.
        velocidade_inicial (float): A velocidade inicial do veículo.
    """
    with Solver(name="z3") as solvedor:
        estados = [declarar_variaveis(i) for i in range(limite)]
        solvedor.add_assertion(inicializar_estado(estados[0], velocidade_inicial))

        for i in range(limite - 1):
            solvedor.add_assertion(definir_transicao(estados[i], estados[i + 1]))

        for i in range(limite):
            solvedor.push()
            solvedor.add_assertion(Not(propriedade(estados[i])))

            if solvedor.solve():
                solvedor.pop()
            else:
                print(f"> A propriedade ocorre em {limite} estados")
                return

        print(f"> A propriedade não ocorre em {limite} estados")
        return

def propriedade_paragem_tempo_limite(estado, limite_tempo):
    """
    Define a propriedade de que o veículo está parado dentro de um limite de tempo.

    Args:
        estado (dict): O dicionário de variáveis de estado.
        limite_tempo (float): O limite de tempo para a paragem.

    Returns:
        FNode: Uma fórmula SMT que representa a propriedade.
    """
    return And(Equals(estado['modo'], MODO_PARADO), estado['tempo'] <= Real(limite_tempo))

def verificacao_bmc_eventualmente_com_tempo(declarar_variaveis, inicializar_estado, definir_transicao, propriedade, limite, limite_tempo, velocidade_inicial):
    """
    Verifica se uma propriedade eventualmente ocorre dentro de um limite usando BMC, com restrição de tempo.

    Args:
        declarar_variaveis (function): Função para declarar variáveis de estado.
        inicializar_estado (function): Função para inicializar o estado.
        definir_transicao (function): Função para definir as transições.
        propriedade (function): Função que define a propriedade a ser verificada.
        limite (int): O limite para o BMC.
        limite_tempo (float): O limite de tempo para a propriedade ocorrer.
        velocidade_inicial (float): A velocidade inicial do veículo.
    """
    with Solver(name="z3") as solvedor:
        estados = [declarar_variaveis(i) for i in range(limite)]
        solvedor.add_assertion(inicializar_estado(estados[0], velocidade_inicial))

        for i in range(limite - 1):
            solvedor.add_assertion(definir_transicao(estados[i], estados[i + 1]))

        for i in range(limite):
            solvedor.push()
            solvedor.add_assertion(Not(propriedade_paragem_tempo_limite(estados[i], limite_tempo)))  # Usa a propriedade com limite de tempo

            if solvedor.solve():
                solvedor.pop()
            else:
                print(f"> Veículo imobiliza-se em menos de {limite_tempo} segundos dentro de {limite} estados")
                return

        print(f"> Veículo não imobiliza-se em menos de {limite_tempo} segundos dentro de {limite} estados")
        return

def verificar_velocidade_a_diminuir(declarar_variaveis, inicializar_estado, definir_transicao, limite, velocidade_inicial):
    """
    Verifica a propriedade de que a velocidade está sempre a diminuir usando BMC.

    Args:
        declarar_variaveis (function): Função para declarar variáveis de estado.
        inicializar_estado (function): Função para inicializar o estado.
        definir_transicao (function): Função para definir as transições.
        limite (int): O limite para o BMC.
        velocidade_inicial (float): A velocidade inicial do veículo.
    """
    with Solver(name="z3") as solvedor:
        estados = [declarar_variaveis(i) for i in range(limite)]
        solvedor.add_assertion(inicializar_estado(estados[0], velocidade_inicial))

        for i in range(limite - 1):
            solvedor.add_assertion(definir_transicao(estados[i], estados[i + 1]))

        for i in range(limite - 1):
            solvedor.add_assertion(Not(propriedade_velocidade_sempre_a_diminuir(estados[i], estados[i + 1])))
            if solvedor.solve():
                print(f"Contradição! A propriedade de diminuição constante de velocidade não se verifica em {i + 1} estados")
                return

        print(f"A propriedade de diminuição constante de velocidade verifica-se em {limite} estados")

# Seção: Testes
gerar_traco(declarar_variaveis, inicializar_estado, definir_transicao, 30, 10)
verificacao_kinducao_sempre(declarar_variaveis, inicializar_estado, definir_transicao, propriedade_limite_temporizador, 30, 10)
verificacao_bmc_sempre(declarar_variaveis, inicializar_estado, definir_transicao, propriedade_limite_temporizador, 30, 10)
verificacao_bmc_eventualmente(declarar_variaveis, inicializar_estado, definir_transicao, propriedade_paragem, 50, 10)
verificacao_bmc_eventualmente_com_tempo(declarar_variaveis, inicializar_estado, definir_transicao, propriedade_paragem, 50, 10.0, 10)
verificar_velocidade_a_diminuir(declarar_variaveis, inicializar_estado, definir_transicao, 50, 10)

is sat
ESTADO: 0
MODO: INICIAL
Velocidade: 10.0
Velocidade da Roda: 10.0
Tempo: 0.0
Temporizador: 0.0


ESTADO: 1
MODO: LIVRE
Velocidade: 10.0
Velocidade da Roda: 10.0
Tempo: 0.0
Temporizador: 0.0


ESTADO: 2
MODO: LIVRE
Velocidade: 9.999
Velocidade da Roda: 9.0
Tempo: 0.1
Temporizador: 0.1


ESTADO: 3
MODO: TRAVAGEM
Velocidade: 9.999
Velocidade da Roda: 9.0
Tempo: 0.1
Temporizador: 0.0


ESTADO: 4
MODO: TRAVAGEM
Velocidade: 8.999
Velocidade da Roda: 8.999
Tempo: 0.2
Temporizador: 0.0


ESTADO: 5
MODO: BLOQUEADO
Velocidade: 8.999
Velocidade da Roda: 8.999
Tempo: 0.2
Temporizador: 0.0


ESTADO: 6
MODO: BLOQUEADO
Velocidade: 8.098
Velocidade da Roda: 8.098
Tempo: 0.30000000000000004
Temporizador: 0.1


ESTADO: 7
MODO: LIVRE
Velocidade: 8.098
Velocidade da Roda: 8.098
Tempo: 0.30000000000000004
Temporizador: 0.0


ESTADO: 8
MODO: LIVRE
Velocidade: 8.097
Velocidade da Roda: 7.098
Tempo: 0.4
Temporizador: 0.1


ESTADO: 9
MODO: TRAVAGEM
Velocidade: 8.097
Velocidade da Roda: 7.098
Tempo: 0.4
